<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [298]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [299]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [300]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [301]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [302]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\elere\AppData\Local\Temp\ipykernel_25108\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_25108\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_25108\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [303]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [304]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'])
apps = apps.drop_duplicates(subset='App', keep="first")

In [305]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [306]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [307]:
DataProcessing.normalization(apps)

In [308]:
apps = apps.drop_duplicates(subset='App', keep="first")

In [309]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Word,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade
0,Sway Medical,1.000,4.455164e-08,0.021148,9.900000e-08,Everyone,Medical,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FC Barcelona Official App,0.900,2.060981e-03,0.055388,4.999999e-03,Everyone,Sports,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Real time Weather Forecast,0.850,9.317976e-05,0.009063,9.999990e-04,Everyone,Weather,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,K Health,0.775,6.437713e-06,0.039275,4.999900e-05,Teen,Health & Fitness,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,mobileRx Pharmacy,0.375,2.739926e-06,0.042296,9.999000e-06,Everyone,Medical,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0
5,B@dL!bs Lite,0.700,2.004824e-07,0.035247,9.990000e-07,Mature 17+,Word,0.432857,0,0,...,1,0,0,0,0,0,0,0,0,0
6,Auction Wars : Storage King,0.775,6.473577e-04,0.062437,9.999990e-04,Everyone,Casino,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0


In [310]:
apps["Content Rating"].unique().tolist()

['Everyone',
 'Everyone 10+',
 'Mature 17+',
 'Teen',
 'Unrated',
 'Adults only 18+']

In [311]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [312]:
validatingSet.sort_values(by=['Content Rating'])

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Word,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade
0,Sway Medical,1.000,4.455164e-08,0.021148,9.900000e-08,Adults only 18+,Medical,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FC Barcelona Official App,0.900,2.060981e-03,0.055388,4.999999e-03,Adults only 18+,Sports,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Real time Weather Forecast,0.850,9.317976e-05,0.009063,9.999990e-04,Adults only 18+,Weather,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,K Health,0.775,6.437713e-06,0.039275,4.999900e-05,Adults only 18+,Health & Fitness,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,mobileRx Pharmacy,0.375,2.739926e-06,0.042296,9.999000e-06,Adults only 18+,Medical,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0
5,B@dL!bs Lite,0.700,2.004824e-07,0.035247,9.990000e-07,Adults only 18+,Word,0.432857,0,0,...,1,0,0,0,0,0,0,0,0,0
6,Auction Wars : Storage King,0.775,6.473577e-04,0.062437,9.999990e-04,Adults only 18+,Casino,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0


In [313]:
trainingSet.loc[trainingSet['App'] == 'BURGER KING® App']

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Word,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade
6662,BURGER KING® App,0.75,0.00093,0.038268,0.005,Everyone,Lifestyle,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2>k-nearest neighbors algorithm</h2>

In [314]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

<h4>Method 1</h4>

In [315]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNN.clustering(trainingSet.copy(), avgApp.iloc[0])
recommendedApps.sort_values("distance").head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade,distance
5233,AnatomyMapp,0.775,1.759790e-06,0.047331,4.999000e-06,Everyone,Medical,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0.782552
6273,CT Abdomen Pelvis,0.750,7.796538e-07,0.035247,9.999000e-06,Everyone,Medical,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0.782670
6227,Super Hearing Super Ear Amplifier,0.775,4.455164e-07,0.019134,9.990000e-07,Everyone,Medical,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0.782718
4229,Prestige AH,0.750,2.227582e-08,0.028197,9.900000e-08,Everyone,Medical,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0.782725
522,Ebenezer AH,0.750,6.682747e-08,0.026183,4.990000e-07,Everyone,Medical,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0.782752


<h4>Method 2</h4>

In [316]:
KNN.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade,distance
3396,CT Scan Viewer 3D,0.800,1.113791e-07,0.028197,9.990000e-07,Everyone,Medical,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.679945
6025,ASCCP Mobile,0.875,1.381101e-06,0.024169,9.999000e-06,Everyone,Medical,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.679960
6422,Deep Sleep and Relax Hypnosis,0.800,1.080600e-04,0.033233,9.999900e-05,Everyone,Medical,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.679970
2660,CT Sessions,0.825,1.358825e-06,0.019134,4.999000e-06,Everyone,Medical,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.680026
69,MbH BP,0.675,1.782066e-07,0.022155,9.990000e-07,Everyone,Medical,0.190000,0,0,...,0,0,0,0,0,0,0,0,0,0.680057
235,PTA Content Master,0.800,1.403377e-06,0.040282,9.990000e-07,Everyone,Medical,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.680118
5233,AnatomyMapp,0.775,1.759790e-06,0.047331,4.999000e-06,Everyone,Medical,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0.680262
5824,Interactive CT & MRI Anat.Lite,0.700,1.187301e-05,0.013092,9.999900e-05,Everyone,Medical,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.680268
1784,FK,0.850,3.377015e-05,0.007553,9.999900e-05,Everyone,Medical,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.680366
1552,Cerebral Palsy,0.850,8.242054e-07,0.007553,9.990000e-07,Everyone,Medical,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0.680366


<h2>Importing Reviews Database</h2>

In [317]:
reviews = pd.read_csv(r"processedReviews.csv")

In [318]:
display(reviews)

,App,reviews
0,10 Best Foods for You,best;food;health;healthy;love
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy;job;use;useful;mobile
2,11st,for;english;there;language;if
3,1800 Contacts - Lens Store,easy;contacts;order;super;love
4,1LINE – One Line with One Touch,game;play;love;ads;many
...,...,...
860,Hotels.com: Book Hotel Rooms & Find Vacation D...,hotel;book;pet;friendly;find
861,Hotspot Shield Free VPN Proxy & Wi-Fi Security,trial;days;free;asking;credit
862,Hotstar,shows;answer;time;u;playing
863,Hotwire Hotel & Car Rental App,would;hotels;hotel;better;card


In [319]:
appsWithReviews = pd.merge(apps, reviews, on="App")

In [320]:
display(appsWithReviews)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade,reviews
0,Birds Sounds Ringtones & Wallpapers,0.900,0.000113,0.022155,0.0010,Everyone 10+,Personalization,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,birds;sounds;sound;please;tones
1,Dresses Ideas & Fashions +3000,0.875,0.000011,0.007251,0.0001,Mature 17+,Beauty,0.085714,0,0,...,0,0,0,0,0,0,0,0,0,dress;see;love;ive;want
2,FamilySearch Tree,0.825,0.000390,0.016113,0.0010,Everyone,Books & Reference,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,family;cant;love;search;even
3,BetterMe: Weight Loss Workouts,0.800,0.000328,0.014099,0.0050,Everyone,Health & Fitness,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,love;work;exercises;trial;would
4,All Video Downloader 2018,0.825,0.000168,0.004632,0.0010,Everyone,Video Players & Editors,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,videos;best;love;super;grand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,DraftKings - Daily Fantasy Sports,0.875,0.001114,0.040282,0.0010,Adults only 18+,Sports,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,love;money;easy;draftkings;fun
618,ExDialer - Dialer & Contacts,0.800,0.002790,0.001712,0.0100,Everyone,Communication,0.157143,0,0,...,0,0,0,0,0,0,0,0,0,dialer;call;phone;stock;contacts
619,10 Best Foods for You,0.750,0.000055,0.002820,0.0005,Everyone 10+,Health & Fitness,0.190000,0,0,...,0,0,0,0,0,0,0,0,0,best;food;health;healthy;love
620,File Browser by Astro (File Manager),0.825,0.013570,0.008258,0.0500,Everyone,Productivity,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,need;phone;ive;time;memory


In [321]:
trainingSet, validatingSet = DataProcessing.split(appsWithReviews, 0.995)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Music & Video,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade,reviews
0,10 Best Foods for You,0.750,0.000055,0.002820,0.0005,Everyone 10+,Health & Fitness,0.190000,0,0,...,0,0,0,0,0,0,0,0,0,best;food;health;healthy;love
1,File Browser by Astro (File Manager),0.825,0.013570,0.008258,0.0500,Everyone,Productivity,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,need;phone;ive;time;memory
2,Bubble Shooter 2,0.825,0.000512,0.036395,0.0050,Everyone,Arcade,0.442857,0,0,...,0,0,0,0,0,0,0,0,1,game;fun;connect;ive;able


In [322]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [323]:
class KNNWithJaccardSimilarityCoefficient:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)-1):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return (sum**(1/2)) * KNNWithJaccardSimilarityCoefficient.jaccard(v1[len(v1)-1], v2[len(v1)-1])
    
    def jaccard(string1,string2):
        sim = 0
        try:
            string1 = string1.split(";")
        except:
            string1 = [""]       
        try:
            string2 = string2.split(";")
        except:
            string2 = [""]
            
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNNWithJaccardSimilarityCoefficient.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNNWithJaccardSimilarityCoefficient.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h4>Method 1</h4>

In [324]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:-1]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
revs = []
for i in validatingSet["reviews"].tolist():
    i = i.split(";")
    for j in i:
        revs.append(j)
words = {}
for word in revs:
        words[word] = 0
for word in revs:
        words[word] +=1
revs = []
for i in range(5):
    try:
        revs.append(max(words, key=words.get))
        del words[max(words, key=words.get)]
    except:
        pass
avgWords = ";".join(revs)
avgAppList.append(avgWords)

avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNNWithJaccardSimilarityCoefficient.clustering(trainingSet.copy(), avgApp.iloc[0])
recommendedApps.sort_values("distance").head(10)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade,reviews,distance
319,Food Calorie Calculator,0.800,0.000029,0.003021,0.0001,Everyone,Health & Fitness,0.428571,0,0,...,0,0,0,0,0,0,0,0,food;no;phone;completely;calorie,0.653454
518,Health and Nutrition Guide,0.825,0.000176,0.002316,0.0005,Everyone,Health & Fitness,0.428571,0,0,...,0,0,0,0,0,0,0,0,helpful;information;informative;health;much,0.653761
144,Calendar+ Schedule Planner App,0.750,0.000200,0.004733,0.0010,Everyone,Productivity,0.432857,0,0,...,0,0,0,0,0,0,0,0,still;love;calendar;updated;ive,0.654641
382,Daily Yoga - Yoga Fitness Plans,0.850,0.001576,0.034240,0.0050,Everyone,Health & Fitness,0.428571,0,0,...,0,0,0,0,0,0,0,0,ive;yoga;love;free;pay,0.654681
58,Hacker's Keyboard,0.850,0.000923,0.000201,0.0010,Everyone,Productivity,0.171429,0,0,...,0,0,0,0,0,0,0,0,keyboard;keys;would;key;best,0.654697
294,Fat Burning Workout - Home Weight lose,0.875,0.000016,0.008459,0.0001,Everyone,Health & Fitness,0.442857,0,0,...,0,0,0,0,0,0,0,0,ads;exercises;time;work;ive,0.656150
138,"Eve Period Tracker - Love, Sex & Relationships...",0.900,0.000453,0.027190,0.0010,Teen,Health & Fitness,0.442857,0,0,...,0,0,0,0,0,0,0,0,love;helpful;best;tracking;easy,0.658299
497,Calorie Counter - EasyFit free,0.925,0.001120,0.010070,0.0010,Everyone,Health & Fitness,0.442857,0,0,...,0,0,0,0,0,0,0,0,food;calories;easy;track;weight,0.660978
539,Home Workout for Men - Bodybuilding,0.950,0.000283,0.014099,0.0010,Everyone,Health & Fitness,0.428571,0,0,...,0,0,0,0,0,0,0,0,best;exercise;workout;see;much,0.664287
195,Calendar Widget Month + Agenda,0.800,0.001355,0.000906,0.0050,Everyone,Productivity,0.432857,0,0,...,0,0,0,0,0,0,0,0,my;problem;already;purchased;cannot,0.816751


<h4>Method 2</h4>

In [325]:
KNNWithJaccardSimilarityCoefficient.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Pretend Play,...,Tools,Simulation,Education,Music & Audio,Business,Educational,Puzzle,Arcade,reviews,distance
58,Hacker's Keyboard,0.850,0.000923,0.000201,0.0010,Everyone,Productivity,0.171429,0,0,...,0,0,0,0,0,0,0,0,keyboard;keys;would;key;best,0.868706
263,Eat Fit - Diet and Health Free,0.775,0.000055,0.002618,0.0001,Everyone,Health & Fitness,0.185714,0,0,...,0,0,0,0,0,0,0,0,error;get;weight;love;configuration,0.950521
271,Advanced Task Killer,0.850,0.012854,0.000504,0.0500,Everyone,Productivity,0.190000,0,0,...,0,0,0,0,0,0,0,0,update;phone;used;s;android,0.951758
101,Block Puzzle,0.900,0.001332,0.006848,0.0050,Everyone,Puzzle,0.185714,0,0,...,0,0,0,0,0,0,1,0,game;time;fun;play;love,1.040403
410,Block Puzzle Classic Legend !,0.800,0.000380,0.003927,0.0050,Everyone,Puzzle,0.190000,0,0,...,0,0,0,0,0,0,1,0,game;time;love;enjoy;interesting,1.132035
205,File Explorer,0.775,0.000022,0.724068,0.0001,Everyone,Productivity,0.171429,0,0,...,0,0,0,0,0,0,0,0,much;es;system;best;wouldnt,1.187643
349,Fancy Widgets,0.800,0.000829,0.002920,0.0050,Everyone,Personalization,0.085714,0,0,...,0,0,0,0,0,0,0,0,love;widget;clock;weather;time,1.226363
558,Four In A Line,0.700,0.000494,0.002014,0.0010,Everyone,Board,0.185714,0,0,...,0,0,0,0,0,0,0,0,game;play;fun;level;easy,1.229278
126,ABC Preschool Free,0.700,0.000614,0.024169,0.0050,Everyone,Education;Education,0.185714,0,0,...,0,0,1,0,0,0,0,0,love;letters;game;brother;kids,1.229541
89,Farm Heroes Saga,0.850,0.169611,0.069486,0.1000,Everyone,Casual,0.185714,0,0,...,0,0,0,0,0,0,0,0,game;level;gold;bars;love,1.237981


<h2>KD Tree Algorithm</h2>

In [326]:
class Node:
    def __init__(self, point, left=None, right=None, name=None):
        self.point = point
        self.left = left
        self.right = right
        self.name = name


class KDTree:
    def __init__(self, points):
        self.k = 4  # Number of dimensions
        self.root = self.build_tree(points)

    def build_tree(self, points, depth=0):
        if len(points) == 0:
            return None

        # Select axis based on depth
        axis = depth % self.k

        # Sort points based on the axis
        sorted_points = points.sort_values(by=points.columns[axis+2])

        # Find the median
        median_index = len(sorted_points) // 2
        median = sorted_points.iloc[median_index]
        
        # Recursively build subtrees
        left_points = sorted_points[:median_index]
        right_points = sorted_points[median_index + 1:]

        left = self.build_tree(left_points, depth + 1)
        right = self.build_tree(right_points, depth + 1)
        return Node(median[1:5], left, right, median[0])

    def find_nearest_neighbors(self, query_point, k=1):
        nearest_points = []
        distances = []
        nearest_names = []

        def search(node, depth=0):
            if node is None:
                return

            # Select axis based on depth
            axis = depth % self.k

            # Traverse left or right subtree based on query point
            if query_point[axis] < node.point[axis]:
                search(node.left, depth + 1)
            else:
                search(node.right, depth + 1)
            # Calculate distance between query point and current point
            distance = np.linalg.norm(np.array(query_point) - np.array(node.point))
            # If the number of nearest points is less than k, add current point
            if len(nearest_points) == 0:
                distances.append(distance)
                nearest_points.append(node.point)
                nearest_names.append(node.name)
            
            elif len(nearest_points) < k:
                index = len(nearest_points)
                for i in range(0, len(nearest_points)):
                    if(distance < distances[i]):
                        index = i
                        break
                distances.insert(index, distance)
                nearest_points.insert(index, node.point)
                nearest_names.insert(index, node.name)
            else:
                # If the current point is closer than the farthest nearest point,
                # replace the farthest point with the current point
                if distance < distances[-1]:
                    for i in range(0, k):
                        if(distance < distances[i]):
                            distances.insert(i, distance)
                            nearest_points.insert(i, node.point)
                            nearest_names.insert(i, node.name)
                            break
                    distances.pop()
                    nearest_points.pop()
                    nearest_names.pop()
            # Check if there can be closer points on the other side of the splitting axis
            if abs(query_point[axis] - node.point[axis]) < distances[-1] or len(nearest_points) < k:
                if query_point[axis] < node.point[axis]:
                    search(node.right, depth + 1)
                else:
                    search(node.left, depth + 1)
        search(self.root)
        return nearest_names

<h2>Building the KD Tree</h2>

In [327]:
kdtree = KDTree(trainingSet)

In [328]:
#average app
query_point = avgApp[['Rating', 'Reviews', 'Size', 'Installs']].to_numpy()[0]

nearest_neighbors = kdtree.find_nearest_neighbors(query_point, k=5)

print("Nearest neighbors:")
for neighbor in nearest_neighbors:
    print(neighbor)

Nearest neighbors:
BetterMe: Weight Loss Workouts
BET NOW - Watch Shows
AC - Tips & News for Android™
Doctor Pets
Etta Homes
